# Loading packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

from os import path
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model, Input

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 32

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Loading datasets

In [5]:
dataset = np.load("resnet_output.npy")
dataset.shape

(50000, 2048)

In [6]:
filenames = list(os.walk(path.join(os.getcwd(), '..', 'cifar-10', 'train', 'train_images'), topdown=False))[0][2]
filenames = pd.DataFrame(filenames, columns=["name"]).reset_index(names="real_index")
filenames

,real_index,name
0,0,1.png
1,1,10.png
2,2,100.png
3,3,1000.png
4,4,10000.png
...,...,...
49995,49995,9995.png
49996,49996,9996.png
49997,49997,9997.png
49998,49998,9998.png


In [7]:
train_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'train.csv')).drop(["Unnamed: 0"], axis=1)
train_labels_df = train_labels_df.merge(filenames, on=["name"])
train_labels_df

,name,codes,real_index
0,1.png,6,0
1,100.png,1,2
2,10001.png,1,5
3,10002.png,6,6
4,10003.png,6,7
...,...,...,...
34995,9994.png,1,49994
34996,9996.png,3,49996
34997,9997.png,9,49997
34998,9998.png,1,49998


In [8]:
val_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'val.csv')).drop(["Unnamed: 0"], axis=1)
val_labels_df = val_labels_df.merge(filenames, on=["name"])
val_labels_df

,name,codes,real_index
0,10.png,3,1
1,1000.png,5,3
2,10000.png,5,4
3,10014.png,6,19
4,10029.png,7,35
...,...,...,...
4995,9961.png,8,49958
4996,9977.png,3,49975
4997,998.png,1,49978
4998,9986.png,0,49985


In [9]:
test_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'test.csv')).drop(["Unnamed: 0"], axis=1)
test_labels_df = test_labels_df.merge(filenames, on=["name"])
test_labels_df

,name,codes,real_index
0,10008.png,6,12
1,1001.png,9,14
2,10012.png,3,17
3,10015.png,5,20
4,10021.png,0,27
...,...,...,...
9995,9971.png,3,49969
9996,9981.png,6,49980
9997,9983.png,3,49982
9998,9984.png,0,49983


In [10]:
assert not train_labels_df.real_index.isin(test_labels_df).any()
assert not train_labels_df.real_index.isin(val_labels_df).any()
assert not test_labels_df.real_index.isin(val_labels_df).any()

In [11]:
train_x, train_y = dataset[train_labels_df.real_index, :], tf.one_hot(train_labels_df.codes, N_CLASS)
val_x, val_y = dataset[val_labels_df.real_index, :], tf.one_hot(val_labels_df.codes, N_CLASS)
test_x, test_y = dataset[test_labels_df.real_index, :], tf.one_hot(test_labels_df.codes, N_CLASS)
train_x.shape, val_x.shape, test_x.shape

((35000, 2048), (5000, 2048), (10000, 2048))

# Resnet FC

In [12]:
def create_model():
    input_layer = Input(shape=(2048))
    x = layers.Dense(2048, activation="relu")(input_layer)
    x = layers.BatchNormalization()(x)
    output = layers.Dense(N_CLASS, activation="softmax")(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 4,225,034
Trainable params: 4,220,938
Non-trainable params: 4,096
_________________________________________________________________


In [13]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
)

checkpoint = callbacks.ModelCheckpoint(
    filepath="resnet_end.h5",
    monitor="val_categorical_accuracy",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, min_lr=0.00001, verbose=1)

In [14]:
history = model.fit(
    x=train_x, y=train_y,
    epochs=200,
    batch_size=256,
    validation_data=(val_x, val_y),
    shuffle=True,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/200
137/137 [==============================] - 2s 7ms/step - loss: 0.4352 - categorical_accuracy: 0.8703 - categorical_crossentropy: 0.4352 - val_loss: 0.3940 - val_categorical_accuracy: 0.8748 - val_categorical_crossentropy: 0.3940 - lr: 0.0010
Epoch 2/200
137/137 [==============================] - 1s 6ms/step - loss: 0.2236 - categorical_accuracy: 0.9219 - categorical_crossentropy: 0.2236 - val_loss: 0.3395 - val_categorical_accuracy: 0.8848 - val_categorical_crossentropy: 0.3395 - lr: 0.0010
Epoch 3/200
137/137 [==============================] - 1s 5ms/step - loss: 0.1725 - categorical_accuracy: 0.9394 - categorical_crossentropy: 0.1725 - val_loss: 0.3011 - val_categorical_accuracy: 0.8980 - val_categorical_crossentropy: 0.3011 - lr: 0.0010
Epoch 4/200
137/137 [==============================] - 1s 5ms/step - loss: 0.1324 - categorical_accuracy: 0.9542 - categorical_crossentropy: 0.1324 - val_loss: 0.2946 - val_categorical_accuracy: 0.9030 - val_categorical_crossentropy: 0.29

# Experiments

Training will be repeated 10 times with different weights initialization.

In [15]:
TRAINING_SEEDS = list(range(10))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = create_model()

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
    )

    history = model.fit(
        x=train_x, y=train_y,
        epochs=200,
        batch_size=256,
        validation_data=(val_x, val_y),
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )
    eval_results = model.evaluate(x=test_x, batch_size=256, y=test_y)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('resnet_results.csv')

Epoch 1/200
137/137 [==============================] - 2s 8ms/step - loss: 0.4373 - categorical_accuracy: 0.8693 - categorical_crossentropy: 0.4373 - val_loss: 0.3497 - val_categorical_accuracy: 0.8858 - val_categorical_crossentropy: 0.3497 - lr: 0.0010
Epoch 2/200
137/137 [==============================] - 1s 6ms/step - loss: 0.2212 - categorical_accuracy: 0.9217 - categorical_crossentropy: 0.2212 - val_loss: 0.3134 - val_categorical_accuracy: 0.8964 - val_categorical_crossentropy: 0.3134 - lr: 0.0010
Epoch 3/200
137/137 [==============================] - 1s 6ms/step - loss: 0.1690 - categorical_accuracy: 0.9403 - categorical_crossentropy: 0.1690 - val_loss: 0.3218 - val_categorical_accuracy: 0.8936 - val_categorical_crossentropy: 0.3218 - lr: 0.0010
Epoch 4/200
137/137 [==============================] - 1s 6ms/step - loss: 0.1380 - categorical_accuracy: 0.9512 - categorical_crossentropy: 0.1380 - val_loss: 0.3338 - val_categorical_accuracy: 0.8932 - val_categorical_crossentropy: 0.33

In [16]:
results

,seed,loss,categorical_accuracy,categorical_crossentropy
0,0,0.321699,0.9230,0.321699
1,1,0.345969,0.9231,0.345969
2,2,0.334492,0.9244,0.334492
3,3,0.353584,0.9241,0.353584
4,4,0.389257,0.9241,0.389257
5,5,0.299078,0.9241,0.299078
6,6,0.341204,0.9256,0.341204
7,7,0.353355,0.9230,0.353355
8,8,0.331920,0.9229,0.331920
9,9,0.355810,0.9198,0.355810
